In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import os
import random
from torch.autograd import Variable
import copy
from torch import nn, optim
import torch.optim as optim
import numpy as np
from torch.autograd import Variable
from collections import OrderedDict
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import csv
import time
import math


In [2]:
def fix_seed(seed):
    # random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

SEED = 42
fix_seed(SEED)

In [3]:
class Argments():
  def __init__(self):
    self.batch_size = 20
    self.test_batch = 1000
    self.global_epochs = 300
    self.local_epochs = 2
    self.lamda = 15
    self.K = 5
    self.lr = None
    self.momentum = 0.9
    self.weight_decay = 10**-4.0
    self.clip = 20.0
    self.partience = 300
    self.worker_num = 20
    self.sample_num = 20
    self.device = device = torch.device('cuda:0'if torch.cuda.is_available() else'cpu')
    self.criterion = nn.CrossEntropyLoss()

args = Argments()

In [4]:
lr = 0

In [6]:
lr_list = []
lr_list.append(10**-3.0)
lr_list.append(10**-2.5)
lr_list.append(10**-2.0)
lr_list.append(10**-1.5)
lr_list.append(10**-1.0)
lr_list.append(10**-0.5)
lr_list.append(10**0.0)
lr_list.append(10**0.5)

args.lr = lr_list[lr]

In [9]:
class LocalDataset(torch.utils.data.Dataset):
  def __init__(self,dataset,worker_id):
    self.data = []
    self.target = []
    self.id = worker_id
    for data in dataset.take(len(dataset)):
      self.data.append(torch.tensor([data['pixels'].numpy()]))
      self.target.append(torch.tensor(data['label'].numpy().astype(np.int64)))

  def __getitem__(self, index):
    return self.data[index],self.target[index]

  def __len__(self):
    return len(self.data)

In [10]:
with open('../data/federated_trainset_femnist.pickle', 'rb') as f:
    all_federated_trainset = pickle.load(f)
with open('../data/federated_testset_femnist.pickle', 'rb') as f:
    all_federated_testset = pickle.load(f)
all_worker_num = len(all_federated_trainset)

In [11]:
worker_id_list = random.sample(range(all_worker_num),args.worker_num)
print(worker_id_list)
federated_trainset = []
federated_testset = []
for i in worker_id_list:
    federated_trainset.append(all_federated_trainset[i])
    federated_testset.append(all_federated_testset[i])

[2619, 456, 102, 3037, 1126, 1003, 914, 571, 3016, 419, 2771, 3033, 2233, 356, 2418, 1728, 130, 122, 383, 895]


In [12]:
federated_valset = [None]*args.worker_num
for i in range(args.worker_num):
  n_samples = len(federated_trainset[i])
  if n_samples==1:
    federated_valset[i] = copy.deepcopy(federated_trainset[i])
  else:
    train_size = int(len(federated_trainset[i]) * 0.7) 
    val_size = n_samples - train_size 
    federated_trainset[i],federated_valset[i] = torch.utils.data.random_split(federated_trainset[i], [train_size, val_size])

In [14]:
class CNN1(torch.nn.Module):
    def __init__(self):
        super(CNN1, self).__init__()
        self.conv2d_1 = torch.nn.Conv2d(1, 32, kernel_size=3)
        self.max_pooling = nn.MaxPool2d(2, stride=2)
        self.dropout_1 = nn.Dropout(0.25)
        self.flatten = nn.Flatten()
        self.linear_1 = nn.Linear(5408, 128)
        self.dropout_2 = nn.Dropout(0.5)
        self.linear_2 = nn.Linear(128,62)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.max_pooling(x)
        x = self.dropout_1(x)
        x = self.flatten(x)
        x = self.relu(self.linear_1(x))
        x = self.dropout_2(x)
        x = self.linear_2(x)
        return x

class CNN2(torch.nn.Module):
    def __init__(self):
        super(CNN2, self).__init__()
        self.conv2d_1 = torch.nn.Conv2d(1, 32, kernel_size=3)
        self.max_pooling = nn.MaxPool2d(2, stride=2)
        self.conv2d_2 = torch.nn.Conv2d(32, 64, kernel_size=3)
        self.dropout_1 = nn.Dropout(0.25)
        self.flatten = nn.Flatten()
        self.linear_1 = nn.Linear(9216, 128)
        self.dropout_2 = nn.Dropout(0.5)
        self.linear_2 = nn.Linear(128,62)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.conv2d_2(x)
        x = self.max_pooling(x)
        x = self.dropout_1(x)
        x = self.flatten(x)
        x = self.relu(self.linear_1(x))
        x = self.dropout_2(x)
        x = self.linear_2(x)
        return x

class CNN3(torch.nn.Module):
    def __init__(self):
        super(CNN3, self).__init__()
        self.conv2d_1 = torch.nn.Conv2d(1, 32, kernel_size=3)
        self.max_pooling = nn.MaxPool2d(2, stride=2)
        self.conv2d_2 = torch.nn.Conv2d(32, 64, kernel_size=3)
        self.conv2d_3 = torch.nn.Conv2d(64, 128, kernel_size=3)
        self.dropout_1 = nn.Dropout(0.25)
        self.flatten = nn.Flatten()
        self.linear_1 = nn.Linear(15488, 128)
        self.dropout_2 = nn.Dropout(0.5)
        self.linear_2 = nn.Linear(128,62)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.conv2d_2(x)
        x = self.conv2d_3(x)
        x = self.max_pooling(x)
        x = self.dropout_1(x)
        x = self.flatten(x)
        x = self.relu(self.linear_1(x))
        x = self.dropout_2(x)
        x = self.linear_2(x)
        return x
    
class CNN4(torch.nn.Module):
    def __init__(self):
        super(CNN4, self).__init__()
        self.conv2d_1 = torch.nn.Conv2d(1, 32, kernel_size=3)
        self.max_pooling = nn.MaxPool2d(2, stride=2)
        self.conv2d_2 = torch.nn.Conv2d(32, 64, kernel_size=3)
        self.conv2d_3 = torch.nn.Conv2d(64, 128, kernel_size=3)
        self.conv2d_4 = torch.nn.Conv2d(128, 256, kernel_size=3)
        self.dropout_1 = nn.Dropout(0.25)
        self.flatten = nn.Flatten()
        self.linear_1 = nn.Linear(25600, 128)
        self.dropout_2 = nn.Dropout(0.5)
        self.linear_2 = nn.Linear(128,62)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.conv2d_2(x)
        x = self.conv2d_3(x)
        x = self.conv2d_4(x)
        x = self.max_pooling(x)
        x = self.dropout_1(x)
        x = self.flatten(x)
        x = self.relu(self.linear_1(x))
        x = self.dropout_2(x)
        x = self.linear_2(x)
        return x

In [15]:
class pFedMeOptimizer(torch.optim.Optimizer):
    def __init__(self, params, lr=0.01, lamda=0.1 , mu = 0.001):
        #self.local_weight_updated = local_weight # w_i,K
        if lr < 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))
        defaults = dict(lr=lr, lamda=lamda, mu = mu)
        super(pFedMeOptimizer, self).__init__(params, defaults)
    
    def step(self, local_weight_updated, closure=None):
        loss = None
        if closure is not None:
            loss = closure
        weight_update = local_weight_updated.copy()
        for group in self.param_groups:
            for p, localweight in zip( group['params'], weight_update):
                localweight.data = localweight.data.to(args.device)
                p.data = p.data - group['lr'] * (p.grad.data + group['lamda'] * (p.data - localweight.data) + group['mu']*p.data)
                localweight.data = localweight.data.to('cpu')
        return  group['params'], loss
    
    def update_param(self, local_weight_updated, closure=None):
        loss = None
        if closure is not None:
            loss = closure
        weight_update = local_weight_updated.copy()
        for group in self.param_groups:
            for p, localweight in zip( group['params'], weight_update):
                p.data = localweight.data
        #return  p.data
        return  group['params']

In [17]:
class Server():
  def __init__(self):
    self.model = CNN2()

  def create_worker(self,federated_trainset,federated_valset,federated_testset):
    workers = []
    for i in range(args.worker_num):
      workers.append(Worker(federated_trainset[i],federated_valset[i],federated_testset[i]))
    return workers

  def sample_worker(self,workers):
    sample_worker = []
    sample_worker_num = random.sample(range(args.worker_num),args.sample_num)
    for i in sample_worker_num:
      sample_worker.append(workers[i])
    return sample_worker


  def send_model(self,workers):
    nums = 0
    for worker in workers:
      nums += worker.train_data_num

    for worker in workers:
      worker.aggregation_weight = 1.0*worker.train_data_num/nums
      worker.model = copy.deepcopy(self.model)
      worker.personalized_model = copy.deepcopy(self.model)
      worker.local_model = copy.deepcopy(self.model)

  def aggregate_model(self,workers):   
    new_params = OrderedDict()
    for i,worker in enumerate(workers):
      worker_state = worker.model.state_dict()
      for key in worker_state.keys():
        if i==0:
          new_params[key] = worker_state[key]*worker.aggregation_weight
        else:
          new_params[key] += worker_state[key]*worker.aggregation_weight
    self.model.load_state_dict(new_params)
    
  def send_parameters(self,workers):
    nums = 0
    for worker in workers:
      nums += worker.train_data_num
    for worker in workers:
        worker.aggregation_weight = 1.0*worker.train_data_num/nums
        worker.set_parameters(self.model)

In [18]:
class Worker():
  def __init__(self,trainset,valset,testset):
    self.trainloader = torch.utils.data.DataLoader(trainset,batch_size=args.batch_size,shuffle=True,num_workers=2)
    self.valloader = torch.utils.data.DataLoader(valset,batch_size=args.test_batch,shuffle=False,num_workers=2)
    self.testloader = torch.utils.data.DataLoader(testset,batch_size=args.test_batch,shuffle=False,num_workers=2)
    #self.iter_trainloader = iter(self.trainloader)
    self.model = CNN2()
    self.local_model = copy.deepcopy(list(self.model.parameters()))
    self.persionalized_model = copy.deepcopy(list(self.model.parameters()))
    self.persionalized_model_bar = copy.deepcopy(list(self.model.parameters()))
    self.train_data_num = len(trainset)
    self.test_data_num = len(testset)
    
    self.optimizer = pFedMeOptimizer(self.model.parameters(),lr=args.lr,lamda=args.lamda)
    
  def set_parameters(self, model):
    for old_param, new_param, local_param in zip(self.model.parameters(), model.parameters(), self.local_model):
        old_param.data = new_param.data.clone()
        local_param.data = new_param.data.clone()
    #self.local_weight_updated = copy.deepcopy(self.optimizer.param_groups[0]['params'])
    
  def get_next_train_batch(self):
    try:
        # Samples a new batch for persionalizing
        (X, y) = next(self.iter_trainloader)
    except StopIteration:
        # restart the generator if the previous generator is exhausted.
        self.iter_trainloader = iter(self.trainloader)
        (X, y) = next(self.iter_trainloader)
    return (X.to(args.device), y.to(args.device))    
  '''
  def local_train(self):
    self.model.train()
    optimizer = pFedMeOptimizer(self.model.parameters(),lr=args.lr,lamda=args.lamda)
    for epoch in range(args.local_epochs):
        running_loss = 0.0
        correct = 0
        count = 0
        data,labels = self.get_next_train_batch()
        for i in range(args.K):
            self.model = self.model.to(args.device)
            optimizer.zero_grad()
            outputs = self.model(data)
            loss = args.criterion(outputs,labels)
            running_loss += loss.item()
            predicted = torch.argmax(outputs,dim=1)
            correct += (predicted==labels).sum().item()
            count += len(labels)
            loss.backward()
            #torch.nn.utils.clip_grad_norm_(self.model.parameters(), args.clip)
            self.model = self.model.to('cpu')
            personal_model,_ = optimizer.step(self.local_model)
        
        for new_param,localweight in zip(personal_model,self.local_model.parameters()):
            localweight.data = localweight.data - args.lamda * args.lr * (localweight.data - new_param.data)    
    
    del self.model
    
    update_parameters(self.personalized_model,personal_model)
    return 100.0*correct/count,running_loss/len(self.trainloader)
  '''

  def local_train(self):
    self.model.train() 
    self.model = self.model.to(args.device)       
    for epoch in range(args.local_epochs):
        running_loss = 0.0
        correct = 0
        count = 0
        for (data,labels) in self.trainloader:
            self.model.train()
            data,labels = Variable(data),Variable(labels)
            data,labels = data.to(args.device),labels.to(args.device)
            for k in range(args.K):
                self.optimizer.zero_grad()
                outputs = self.model(data)
                loss = args.criterion(outputs,labels)
                if k==(args.K-1):
                    running_loss += loss.item()
                    predicted = torch.argmax(outputs,dim=1)
                    correct += (predicted==labels).sum().item()
                    count += len(labels)
                loss.backward()
                #torch.nn.utils.clip_grad_norm_(self.model.parameters(), args.clip)
                self.persionalized_model_bar,_ = self.optimizer.step(self.local_model)
        
            for new_param, localweight in zip(self.persionalized_model_bar, self.local_model):
                localweight.data = localweight.data.to(args.device)
                localweight.data = localweight.data - args.lamda* args.lr * (localweight.data - new_param.data)
                localweight.data = localweight.data.to('cpu')
    
    self.update_parameters(self.local_model)
    
    self.model = self.model.to('cpu')
    for personal_weight, localweight in zip(self.persionalized_model_bar, self.local_model):
        personal_weight.data = personal_weight.data.to('cpu')
        localweight.data = localweight.data.to('cpu')
        
    return 100.0*correct/count,running_loss/len(self.trainloader)


  def validate(self):
    self.model.eval()
    self.update_parameters(self.persionalized_model_bar)
    self.model = self.model.to(args.device)
    acc,loss = test(self.model,args.criterion,self.valloader)
    self.model = self.model.to('cpu')
    self.update_parameters(self.local_model)
    return acc,loss


  def test(self):
    self.model.eval()
    self.update_parameters(self.persionalized_model_bar)
    self.model = self.model.to(args.device)
    acc,loss = test(self.model,args.criterion,self.testloader)
    self.model = self.model.to('cpu')
    self.update_parameters(self.local_model)
    return acc,loss


  def update_parameters(self, new_params):
    for param , new_param in zip(self.model.parameters(), new_params):
      param.data = new_param.data.clone()

In [19]:
def train(model,criterion,trainloader,epochs):
  optimizer = optim.SGD(model.parameters(),lr=args.lr,momentum=args.momentum,weight_decay=args.weight_decay)
  model.train()
  for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    count = 0
    for (data,labels) in trainloader:
      data,labels = Variable(data),Variable(labels)
      data,labels = data.to(args.device),labels.to(args.device)
      optimizer.zero_grad()
      outputs = model(data)
      loss = criterion(outputs,labels)
      running_loss += loss.item()
      predicted = torch.argmax(outputs,dim=1)
      correct += (predicted==labels).sum().item()
      count += len(labels)
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
      optimizer.step()

  return 100.0*correct/count,running_loss/len(trainloader)



In [20]:
def test(model,criterion,testloader):
  model.eval()
  running_loss = 0.0
  correct = 0
  count = 0
  for (data,labels) in testloader:
    data,labels = data.to(args.device),labels.to(args.device)
    outputs = model(data)
    running_loss += criterion(outputs,labels).item()
    predicted = torch.argmax(outputs,dim=1)
    correct += (predicted==labels).sum().item()
    count += len(labels)

  accuracy = 100.0*correct/count
  loss = running_loss/len(testloader)


  return accuracy,loss

In [21]:
class Early_Stopping():
  def __init__(self,partience):
    self.step = 0
    self.loss = float('inf')
    self.partience = partience

  def validate(self,loss):
    if self.loss<loss:
      self.step += 1
      if self.step>self.partience:
        return True
    else:
      self.step = 0
      self.loss = loss

    return False

In [22]:
server = Server()
workers = server.create_worker(federated_trainset,federated_valset,federated_testset)
acc_train = []
loss_train = []
acc_valid = []
loss_valid = []

early_stopping = Early_Stopping(args.partience)

start = time.time()#開始時刻

for epoch in range(args.global_epochs):
  sample_worker = server.sample_worker(workers)
  server.send_parameters(sample_worker)

  acc_train_avg = 0.0
  loss_train_avg = 0.0
  acc_valid_avg = 0.0
  loss_valid_avg = 0.0
  for worker in sample_worker:
    acc_train_tmp,loss_train_tmp = worker.local_train()
    acc_valid_tmp,loss_valid_tmp = worker.validate()
    acc_train_avg += acc_train_tmp/len(sample_worker)
    loss_train_avg += loss_train_tmp/len(sample_worker)
    acc_valid_avg += acc_valid_tmp/len(sample_worker)
    loss_valid_avg += loss_valid_tmp/len(sample_worker)
  server.aggregate_model(sample_worker)
  '''
  server.model.to(args.device)
  for worker in workers:
    acc_valid_tmp,loss_valid_tmp = test(server.model,args.criterion,worker.valloader)
    acc_valid_avg += acc_valid_tmp/len(workers)
    loss_valid_avg += loss_valid_tmp/len(workers)
  server.model.to('cpu')
  '''
  print('Epoch{}  loss:{}  accuracy:{}'.format(epoch+1,loss_valid_avg,acc_valid_avg))
  acc_train.append(acc_train_avg)
  loss_train.append(loss_train_avg)
  acc_valid.append(acc_valid_avg)
  loss_valid.append(loss_valid_avg)

  if early_stopping.validate(loss_valid_avg):
    print('Early Stop')
    break
    
end = time.time()#終了時刻

Epoch1  loss:4.13703670501709  accuracy:2.0569695731677236
Epoch2  loss:4.120308327674866  accuracy:2.648791925787419
Epoch3  loss:4.1025719165801995  accuracy:2.6487919257874184
Epoch4  loss:4.086331701278686  accuracy:2.5932363702318635
Epoch5  loss:4.07083797454834  accuracy:2.593236370231863
Epoch6  loss:4.05720729827881  accuracy:2.593236370231862
Epoch7  loss:4.044193887710572  accuracy:2.593236370231863
Epoch8  loss:4.032093775272369  accuracy:2.593236370231863
Epoch9  loss:4.021188199520111  accuracy:2.5932363702318635
Epoch10  loss:4.010584497451781  accuracy:2.593236370231863
Epoch11  loss:3.9999088168144223  accuracy:2.5932363702318626
Epoch12  loss:3.9908419847488403  accuracy:2.5932363702318635
Epoch13  loss:3.982397389411927  accuracy:2.593236370231863
Epoch14  loss:3.9734476804733276  accuracy:2.5932363702318626
Epoch15  loss:3.9653611183166504  accuracy:2.5932363702318635
Epoch16  loss:3.9574843287467956  accuracy:2.5932363702318635
Epoch17  loss:3.9495299696922306  acc

Epoch136  loss:3.7187142968177795  accuracy:10.29275657910491
Epoch137  loss:3.7180507302284242  accuracy:10.63232864500632
Epoch138  loss:3.7168899416923527  accuracy:10.454279632462535
Epoch139  loss:3.716241884231567  accuracy:10.756797343168397
Epoch140  loss:3.715255975723266  accuracy:11.256136892507948
Epoch141  loss:3.71541029214859  accuracy:10.124179419087058
Epoch142  loss:3.7151982665061953  accuracy:10.629057873965515
Epoch143  loss:3.714275693893432  accuracy:11.206999676645992
Epoch144  loss:3.7131517291069027  accuracy:10.85558158665944
Epoch145  loss:3.7128818869590763  accuracy:10.762484203733308
Epoch146  loss:3.7117318630218508  accuracy:10.948619651715445
Epoch147  loss:3.712245678901673  accuracy:11.016951242695121
Epoch148  loss:3.7111969351768495  accuracy:10.820420711669819
Epoch149  loss:3.711075258255005  accuracy:11.51869727057355
Epoch150  loss:3.7107816100120545  accuracy:11.37766766525823
Epoch151  loss:3.7104048848152154  accuracy:11.12342795203503
Epoch

Epoch268  loss:3.6470594763755804  accuracy:12.53760369274567
Epoch269  loss:3.6462381005287168  accuracy:12.561651393012884
Epoch270  loss:3.644699859619141  accuracy:12.180338981389337
Epoch271  loss:3.645521163940429  accuracy:12.739684093511462
Epoch272  loss:3.6445984482765192  accuracy:12.489752492455448
Epoch273  loss:3.643868112564087  accuracy:12.584700092281095
Epoch274  loss:3.6425904035568233  accuracy:12.850878304451598
Epoch275  loss:3.642225015163421  accuracy:12.595111428911943
Epoch276  loss:3.641817915439606  accuracy:12.833370705762524
Epoch277  loss:3.64133540391922  accuracy:12.719443600805091
Epoch278  loss:3.6396702289581295  accuracy:12.675063745186701
Epoch279  loss:3.6392205476760866  accuracy:13.204264131363837
Epoch280  loss:3.6383398294448854  accuracy:12.887080840400056
Epoch281  loss:3.638421654701233  accuracy:13.150824361282384
Epoch282  loss:3.636988544464111  accuracy:13.390767185115877
Epoch283  loss:3.6368739485740664  accuracy:12.919034707372944
Ep

In [23]:
print('学習時間：{}秒'.format(end-start))#終了時刻-開始時刻でかかった時間

学習時間：19069.10147500038秒


In [24]:
acc_test_personalized = []
loss_test_personalized = []

start = time.time()#開始時刻

for i,worker in enumerate(workers):
  acc_tmp,loss_tmp = worker.test()
  acc_test_personalized.append(acc_tmp)
  loss_test_personalized.append(loss_tmp)
  print('Worker{} accuracy:{}  loss:{}'.format(i+1,acc_tmp,loss_tmp))

end = time.time()#終了時刻

acc_test_personalized_avg = sum(acc_test_personalized)/len(acc_test_personalized)
loss_test_personalized_avg = sum(loss_test_personalized)/len(loss_test_personalized)
print('Test(personalized)  loss:{}  accuracy:{}'.format(loss_test_personalized_avg,acc_test_personalized_avg))

Worker1 accuracy:11.764705882352942  loss:3.626681327819824
Worker2 accuracy:22.857142857142858  loss:3.5399868488311768
Worker3 accuracy:15.0  loss:3.5284180641174316
Worker4 accuracy:16.666666666666668  loss:3.752781391143799
Worker5 accuracy:5.882352941176471  loss:3.56485652923584
Worker6 accuracy:23.80952380952381  loss:3.454990863800049
Worker7 accuracy:21.05263157894737  loss:3.6168129444122314
Worker8 accuracy:12.5  loss:3.6710546016693115
Worker9 accuracy:21.05263157894737  loss:3.63863205909729
Worker10 accuracy:12.195121951219512  loss:3.499849319458008
Worker11 accuracy:0.0  loss:3.895113229751587
Worker12 accuracy:31.57894736842105  loss:3.4048798084259033
Worker13 accuracy:15.789473684210526  loss:3.549311637878418
Worker14 accuracy:10.0  loss:3.8056797981262207
Worker15 accuracy:31.57894736842105  loss:3.5425455570220947
Worker16 accuracy:11.764705882352942  loss:3.4685235023498535
Worker17 accuracy:18.181818181818183  loss:3.486377239227295
Worker18 accuracy:16.12903225

In [25]:
print('推論時間：{}秒'.format(end-start))#終了時刻-開始時刻でかかった時間

推論時間：15.378531455993652秒


In [26]:
acc_test_global = []
loss_test_global = []

start = time.time()#開始時刻

for i,worker in enumerate(workers):
  server.model = server.model.to(args.device)
  acc_tmp,loss_tmp = test(server.model,args.criterion,worker.testloader)
  acc_test_global.append(acc_tmp)
  loss_test_global.append(loss_tmp)
  print('Worker{} accuracy:{}  loss:{}'.format(i+1,acc_tmp,loss_tmp))

server.model = server.model.to('cpu')

end = time.time()#終了時刻

acc_test_global_avg = sum(acc_test_global)/len(acc_test_global)
loss_test_global_avg = sum(loss_test_global)/len(loss_test_global)
print('Test(global)  loss:{}  accuracy:{}'.format(loss_test_global_avg,acc_test_global_avg))

Worker1 accuracy:11.764705882352942  loss:3.6528284549713135
Worker2 accuracy:25.714285714285715  loss:3.604044198989868
Worker3 accuracy:15.0  loss:3.5448813438415527
Worker4 accuracy:16.666666666666668  loss:3.775890350341797
Worker5 accuracy:5.882352941176471  loss:3.578462600708008
Worker6 accuracy:14.285714285714286  loss:3.4692904949188232
Worker7 accuracy:10.526315789473685  loss:3.622075080871582
Worker8 accuracy:12.5  loss:3.6884396076202393
Worker9 accuracy:15.789473684210526  loss:3.663530111312866
Worker10 accuracy:4.878048780487805  loss:3.548125743865967
Worker11 accuracy:0.0  loss:3.927037477493286
Worker12 accuracy:21.05263157894737  loss:3.429694652557373
Worker13 accuracy:15.789473684210526  loss:3.557819128036499
Worker14 accuracy:10.0  loss:3.8104965686798096
Worker15 accuracy:21.05263157894737  loss:3.5714282989501953
Worker16 accuracy:11.764705882352942  loss:3.4876160621643066
Worker17 accuracy:3.0303030303030303  loss:3.511375665664673
Worker18 accuracy:12.90322

In [27]:
print('推論時間：{}秒'.format(end-start))#終了時刻-開始時刻でかかった時間

推論時間：14.885394096374512秒


In [28]:
acc_tune_test_global = []
loss_tune_test_global = []

start = time.time()#開始時刻

for i,worker in enumerate(workers):
  worker.model = copy.deepcopy(server.model)
  worker.model = worker.model.to(args.device)
  _,_ = train(worker.model,args.criterion,worker.trainloader,args.local_epochs)
  acc_tmp,loss_tmp = test(worker.model,args.criterion,worker.testloader)
  acc_tune_test_global.append(acc_tmp)
  loss_tune_test_global.append(loss_tmp)
  print('Worker{} accuracy:{}  loss:{}'.format(i+1,acc_tmp,loss_tmp))
  worker.model = worker.model.to('cpu')
  del worker.model

end = time.time()#終了時刻

acc_tune_test_global_avg = sum(acc_tune_test_global)/len(acc_tune_test_global)
loss_tune_test_global_avg = sum(loss_tune_test_global)/len(loss_tune_test_global)
print('Test_fine-tune(global)  loss:{}  accuracy:{}'.format(loss_tune_test_global_avg,acc_tune_test_global_avg))

Worker1 accuracy:5.882352941176471  loss:3.5585062503814697
Worker2 accuracy:2.857142857142857  loss:3.388286590576172
Worker3 accuracy:12.5  loss:3.5617835521698
Worker4 accuracy:5.555555555555555  loss:3.939203977584839
Worker5 accuracy:14.705882352941176  loss:3.6461002826690674
Worker6 accuracy:4.761904761904762  loss:3.4343273639678955
Worker7 accuracy:10.526315789473685  loss:3.684765577316284
Worker8 accuracy:9.375  loss:3.651965379714966
Worker9 accuracy:0.0  loss:3.760240316390991
Worker10 accuracy:12.195121951219512  loss:3.4825937747955322
Worker11 accuracy:5.555555555555555  loss:3.8849945068359375
Worker12 accuracy:5.2631578947368425  loss:3.448474168777466
Worker13 accuracy:0.0  loss:3.7474770545959473
Worker14 accuracy:3.3333333333333335  loss:3.892516613006592
Worker15 accuracy:5.2631578947368425  loss:3.469998359680176
Worker16 accuracy:11.764705882352942  loss:3.4498631954193115
Worker17 accuracy:18.181818181818183  loss:3.50374174118042
Worker18 accuracy:12.903225806

In [29]:
print('学習・推論時間（fine-tune）：{}秒'.format(end-start))#終了時刻-開始時刻でかかった時間

学習・推論時間（fine-tune）：46.49528503417969秒
